In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import time

In [3]:
# 讀取村里界.gpkg (多邊形) - 使用 EPSG:3826 TWD97 / TM2 zone 121
village_gdf = gpd.read_file("村里界.gpkg")
print(f"村里界座標系統: {village_gdf.crs}")

# coords.csv 疊圖  - x,y 使用 EPSG:3826 TWD97 / TM2 zone 121
coords_df = pd.read_csv("coords.csv")
print(f"coords.csv 座標系統: EPSG:3826 TWD97 / TM2 zone 121")
# 將 coords_df 轉換為 GeoDataFrame
coords_gdf = gpd.GeoDataFrame(
    coords_df, 
    geometry=gpd.points_from_xy(coords_df['x'], coords_df['y']),
    crs="EPSG:3826"
)

# 檢查 coords_gdf 的幾何類型
print(f"coords_gdf 幾何類型: {coords_gdf.geom_type.unique()}")

# 檢查 coords_gdf 的座標系統
print(f"coords_gdf 座標系統: {coords_gdf.crs}")

# 進行空間連接
start_time = time.time()
joined_gdf = gpd.sjoin(coords_gdf, village_gdf, how="inner", predicate="intersects")
end_time = time.time()
print(f"空間連接耗時: {end_time - start_time:.4f} 秒")
# 檢查連接結果
print(f"連接結果數量: {len(joined_gdf)}")
# save as CSV
joined_gdf.to_csv("coords_村里.csv", index=False)

村里界座標系統: EPSG:3826
coords.csv 座標系統: EPSG:3826 TWD97 / TM2 zone 121
coords_gdf 幾何類型: ['Point']
coords_gdf 座標系統: EPSG:3826
空間連接耗時: 0.6529 秒
連接結果數量: 61174


In [23]:
# 讀取村里界.gpkg (多邊形) - 使用 EPSG:3826 TWD97 / TM2 zone 121
village_gdf = gpd.read_file("村里界.gpkg")
print(f"村里界座標系統: {village_gdf.crs}")

# 讀取 A1_all.feather，並將經度、緯度轉為點
start_time = time.time()
a1_df = pd.read_feather("A1_all.feather")
# assign first row as header, then drop it
a1_df.columns = a1_df.iloc[0]
a1_df = a1_df[1:]
read_time = time.time()
print(f"讀取 A1_all.feather 耗時: {time.time() - start_time:.2f} 秒")

# 確保經度和緯度是數值型態，並過濾無效值
a1_df['經度'] = pd.to_numeric(a1_df['經度'], errors='coerce')
a1_df['緯度'] = pd.to_numeric(a1_df['緯度'], errors='coerce')

# 移除經度或緯度為空值的資料
a1_df = a1_df.dropna(subset=['經度', '緯度'])

print(f"有效事故資料筆數: {len(a1_df)}")

# 建立事故點 GeoDataFrame，使用 WGS84 座標系統 (經緯度)
a1_gdf = gpd.GeoDataFrame(
    a1_df,
    geometry=[Point(xy) for xy in zip(a1_df['經度'], a1_df['緯度'])],
    crs='EPSG:4326'  # WGS84 地理座標系統
)

# 將事故點座標系統轉換為與村里界相同的 TWD97 / TM2 zone 121
a1_gdf = a1_gdf.to_crs(village_gdf.crs)

# 疊圖：將事故點與村里界進行空間聯集
joined_gdf = gpd.sjoin(a1_gdf, village_gdf, how="left", predicate="within")

time_taken = time.time() - read_time
print(f"疊圖用時: {time_taken:.2f} 秒")

# joined_gdf 現在包含各事故對應的縣市、鄉鎮市區、村里等欄位
# joined_gdf.head()

村里界座標系統: EPSG:3826
讀取 A1_all.feather 耗時: 0.07 秒
有效事故資料筆數: 29142
疊圖用時: 0.56 秒


In [ ]:
# 讀取 A2_all.feather，並將經度、緯度轉為點
start_time = time.time()
a2_df = pd.read_feather("A2_all.feather")
# assign first row as header, then drop it
a2_df.columns = a2_df.iloc[0]
a2_df = a2_df[1:]
read_time = time.time()
print(f"讀取 A2_all.feather 耗時: {time.time() - start_time:.2f} 秒")

# 確保經度和緯度是數值型態，並過濾無效值
a2_df['經度'] = pd.to_numeric(a2_df['經度'], errors='coerce')  
a2_df['緯度'] = pd.to_numeric(a2_df['緯度'], errors='coerce')

# 移除經度或緯度為空值的資料
a2_df = a2_df.dropna(subset=['經度', '緯度'])

print(f"有效事故資料筆數: {len(a2_df)}")

# 建立事故點 GeoDataFrame，使用 WGS84 座標系統 (經緯度)
a2_gdf = gpd.GeoDataFrame(
    a2_df,
    geometry=[Point(xy) for xy in zip(a2_df['經度'], a2_df['緯度'])],
    crs='EPSG:4326'  # WGS84 地理座標系統
)

# 將事故點座標系統轉換為與村里界相同的 TWD97 / TM2 zone 121
a2_gdf = a2_gdf.to_crs(village_gdf.crs)

# 疊圖：將事故點與村里界進行空間聯集
joined_gdf2 = gpd.sjoin(a2_gdf, village_gdf, how="left", predicate="within")

time_taken = time.time() - read_time
print(f"疊圖用時: {time_taken:.2f} 秒")

# joined_gdf2 現在包含各事故對應的縣市、鄉鎮市區、村里等欄位
# joined_gdf2.head()

讀取 A1_all.feather 耗時: 20.05 秒
有效事故資料筆數: 5384845
疊圖用時: 126.01 秒


In [25]:
# save joined data to feather files
start_time = time.time()
joined_gdf.to_feather("A1_joined.feather")
A1_end_time = time.time()
print(f"儲存 A1_joined.feather 耗時: {A1_end_time - start_time:.2f} 秒")
A2_start_time = time.time()
joined_gdf2.to_feather("A2_joined.feather")
A2_end_time = time.time()
print(f"儲存 A2_joined.feather 耗時: {A2_end_time - A2_start_time:.2f} 秒")

儲存 A1_joined.feather 耗時: 0.32 秒
儲存 A2_joined.feather 耗時: 51.02 秒


In [2]:
#read A1_joined.feather and A2_joined.feather
start_time = time.time()
a1_joined_df = pd.read_feather("A1_joined.feather")
A1_joined_end_time = time.time()
print(f"讀取 A1_joined.feather 耗時: {A1_joined_end_time - start_time:.2f} 秒")
A2_joined_start_time = time.time()
a2_joined_df = pd.read_feather("A2_joined.feather")
A2_joined_end_time = time.time()
print(f"讀取 A2_joined.feather 耗時: {A2_joined_end_time - A2_joined_start_time:.2f} 秒")

讀取 A1_joined.feather 耗時: 0.45 秒
讀取 A2_joined.feather 耗時: 46.55 秒


In [3]:
# filter TOWNAME and save as gpkg
def filter_and_save(df, town_name, filename):
    # 先篩選指定城鎮的資料
    filtered_df = df[df['TOWNNAME'] == town_name]
    if not filtered_df.empty:
        # 由於 feather 格式不保存幾何資訊，需要從經緯度重新建立幾何
        # 檢查是否有經緯度欄位
        if '經度' in filtered_df.columns and '緯度' in filtered_df.columns:
            # 從經緯度重新建立 Point 幾何
            filtered_gdf = gpd.GeoDataFrame(
                filtered_df,
                geometry=[Point(xy) for xy in zip(filtered_df['經度'], filtered_df['緯度'])],
                crs='EPSG:4326'  # 原始座標系統 (WGS84)
            )
            # 轉換為 TWD97
            filtered_gdf = filtered_gdf.to_crs('EPSG:3826')
        else:
            print(f"警告：{town_name} 的資料缺少經緯度欄位")
            return
        
        filtered_gdf.to_file(filename, driver='GPKG')
        print(f"已儲存 {town_name} 的資料到 {filename}，共 {len(filtered_gdf)} 筆")
    else:
        print(f"{town_name} 沒有資料")

In [5]:
# filter and save for each town
towns = ['大安區']
for town in towns:
    filter_and_save(a1_joined_df, town, f"A1_{town}.gpkg")
    filter_and_save(a2_joined_df, town, f"A2_{town}.gpkg")

已儲存 大安區 的資料到 A1_大安區.gpkg，共 150 筆
已儲存 大安區 的資料到 A2_大安區.gpkg，共 39280 筆
